In [105]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [106]:
office_pcd = o3d.io.read_point_cloud("data\\fragment.ply")
o3d.visualization.draw_geometries([office_pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])
type(office_pcd)

open3d.cpu.pybind.geometry.PointCloud

In [107]:
point_cloud_np = np.asarray(office_pcd.points)
# Extract X, Y, and Z coordinates
x = point_cloud_np[:, 0]
y = point_cloud_np[:, 1]
z = point_cloud_np[:, 2]
# Print or use the X, Y, and Z coordinates as needed
print('X coordinates:', x)
print('Y coordinates:', y)
print('Z coordinates:', z)
print(len(point_cloud_np))

X coordinates: [0.65234375 0.65234375 0.66737998 ... 2.00839925 2.00390625 2.00390625]
Y coordinates: [0.84686458 0.83984375 0.83984375 ... 2.39453125 2.39488506 2.39453125]
Z coordinates: [2.37890625 2.38430572 2.37890625 ... 1.88671875 1.88671875 1.88793314]
196133


### Just accessing the points and indeces of the queury point 

In [108]:
points = np.vstack((x, y, z)).T
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
kdtree = o3d.geometry.KDTreeFlann(pcd)
                                            ### Queury Point
query_point = np.array([2.00839925, 2.39453125, 1.88671875])
                                            ### Finding the indexes of nearest 5000 points
_, indices, _ = kdtree.search_knn_vector_3d(query_point, 500)

print(len(indices))
print(len(points))
print(type(points))
print(points)


500
196133
<class 'numpy.ndarray'>
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


### getting the indeces of the nearest neighbours

In [109]:
# Getting the coordinates of points using indices
coordinates_to_delete = points[indices]
print(coordinates_to_delete)
print(len(coordinates_to_delete))
print(type(coordinates_to_delete))

[[2.00839925 2.39453125 1.88671875]
 [2.01171875 2.39453125 1.8859278 ]
 [2.01171875 2.39223051 1.88671875]
 ...
 [2.05859375 2.32421875 1.87442505]
 [2.08203125 2.39791465 1.83984375]
 [1.92578125 2.38671875 1.91403759]]
500
<class 'numpy.ndarray'>


In [110]:
# Filter out the points that are not in the 'coordinates_to_delete'
remaining_points = np.array([point for point in points if point not in coordinates_to_delete])

# Convert 'remaining_points' to Open3D point cloud format
remaining_pcd = o3d.geometry.PointCloud()
remaining_pcd.points = o3d.utility.Vector3dVector(remaining_points)

# Print information about remaining points
print(len(remaining_points))
print(type(remaining_points))

# Visualize or save 'remaining_pcd' as needed
o3d.visualization.draw_geometries([remaining_pcd]) 


170099
<class 'numpy.ndarray'>


### Function that Detects Hole and Boundry of the Hole and Point Cloud

In [111]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])

In [113]:
print("Statistical oulier removal")
cl, ind = remaining_pcd.remove_statistical_outlier(nb_neighbors=35,
                                                    std_ratio=1.0)
display_inlier_outlier(remaining_pcd, ind)

Statistical oulier removal
Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Function that generate the Hole

In [115]:
def hole_generation(neighbours, pcd):

    point_cloud_np = np.asarray(pcd.points)
                                            ### getting the x,y and z cordinates of the whole point cloud
    x = point_cloud_np[:, 0]
    y = point_cloud_np[:, 1]
    z = point_cloud_np[:, 2]

    points = np.vstack((x, y, z)).T
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    kdtree = o3d.geometry.KDTreeFlann(pcd)
                                                                    ### Queury Point
    query_point = np.array([2.00839925, 2.39453125, 1.88671875])
                                                      ### Finding the indexes of nearest 5000 points
    _, indices, _ = kdtree.search_knn_vector_3d(query_point, neighbours)

    coordinates_to_delete = points[indices]

    remaining_points = np.array([point for point in points if point not in coordinates_to_delete])

    # Convert 'remaining_points' to Open3D point cloud format
    remaining_pcd = o3d.geometry.PointCloud()
    remaining_pcd.points = o3d.utility.Vector3dVector(remaining_points)

    o3d.visualization.draw_geometries([remaining_pcd]) 

In [116]:
hole_generation(500, office_pcd)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
